In [39]:
import pandas as pd

train_df = pd.read_excel('./bootcap.xlsx', header=None)
test_df = pd.read_excel('./TestData.xlsx', header=None)

X_train = train_df.iloc[:, :-1]  # 特征: 前4列
y_train = train_df.iloc[:, -1]   # 标签: 最后一列

X_test = test_df.iloc[:, :]  # 没有标签

你是对的，这个 `HandmadeDecisionTree` 类实现的是一个基本的二叉决策树，它使用基尼不纯度（Gini impurity）来选择最佳分裂特征和阈值，这是 CART 算法的一部分。它也实现了两个基本的停止条件：最大深度和最小样本数。

然而，这个类并没有实现信息增益（ID3 算法）和增益率（C4.5 算法）的计算，也没有实现纯度提升的最小阈值这个停止条件。此外，它也没有实现预剪枝和后剪枝。

如果你需要这些功能，你可能需要修改 `HandmadeDecisionTree` 类，或者使用一个更完整的决策树实现，比如 scikit-learn 的 `DecisionTreeClassifier`。scikit-learn 的决策树实现了许多高级功能，包括多种分裂质量的度量方法（例如信息增益和基尼不纯度），多种停止条件，以及预剪枝。

In [40]:
import numpy as np

class HandmadeDecisionTree:
    def __init__(self, max_depth=None, min_size=1, min_impurity_decrease=0.0):
        self.max_depth = max_depth
        self.min_size = min_size
        self.min_impurity_decrease = min_impurity_decrease
        self.root = None

    # Calculate the Gini index for a split dataset
    def gini_index(self, groups, classes):
        n_instances = float(sum([len(group) for group in groups]))
        gini = 0.0
        for group in groups:
            size = float(len(group))
            if size == 0:
                continue
            score = 0.0
            for class_val in classes:
                p = [row[-1] for row in group].count(class_val) / size
                score += p * p
            gini += (1.0 - score) * (size / n_instances)
        return gini

    # Split a dataset
    def test_split(self, index, value, dataset):
        left, right = list(), list()
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right

    # Find the best split point given gini index
    def get_split(self, dataset):
        class_values = list(set(row[-1] for row in dataset))
        b_index, b_value, b_score, b_groups = 999, 999, 999, None
        best_gini = 1.0  # initial worst gini
        for index in range(len(dataset[0])-1):
            for row in dataset:
                groups = self.test_split(index, row[index], dataset)
                gini = self.gini_index(groups, class_values)
                if gini < b_score:
                    current_gini = self.gini_index([dataset], class_values)
                    impurity_decrease = current_gini - gini
                    if impurity_decrease > self.min_impurity_decrease:  # apply pre-pruning
                        b_index, b_value, b_score, b_groups = index, row[index], gini, groups
                        best_gini = gini
        if best_gini == 1.0:  # if best gini no improvement, then no split
            return None
        return {'index':b_index, 'value':b_value, 'groups':b_groups}

    # Create a terminal node value
    def to_terminal(self, group):
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count)

    # Create child splits for a node or make terminal
    def split(self, node, max_depth, min_size, depth):
        if node is None:  # if node is none then return
            return
        
        left, right = node.get('groups', (None, None))  # get to prevent KeyError
        if left is None or right is None:  # if left or right is none then
            return
        
        del(node['groups'])
        
        # 检查左右子树是否为空
        if not left or not right:
            node['left'] = node['right'] = self.to_terminal(left + right)
            return
        
        # based on the max_depth and min_size, decide whether to split further
        if depth >= max_depth:
            node['left'], node['right'] = self.to_terminal(left), self.to_terminal(right)
        else:
            if len(left) <= min_size:
                node['left'] = self.to_terminal(left)
            else:
                node['left'] = self.get_split(left)
                self.split(node['left'], max_depth, min_size, depth+1)
            
            if len(right) <= min_size:
                node['right'] = self.to_terminal(right)
            else:
                node['right'] = self.get_split(right)
                self.split(node['right'], max_depth, min_size, depth+1)

    # Build a decision tree
    def build_tree(self, train):
        self.root = self.get_split(train)
        self.split(self.root, self.max_depth, self.min_size, 1)
 
    # Make a prediction with this decision tree
    def predict(self, node, row):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self.predict(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self.predict(node['right'], row)
            else:
                return node['right']
    # Train
    def fit(self, X, y):
        dataset = np.column_stack((X,y))
        self.build_tree(dataset.tolist())

    def predict_row(self, row):
        return self.predict(self.root, row)
    
    # caculate accuracy
    def score(self, X, y):
        correct = 0
        for i in range(len(X)):
            if self.predict_row(X.iloc[i, :]) == y.iloc[i]:
                correct += 1
        return correct / float(len(X))


In [41]:
from sklearn.model_selection import KFold

def cross_validate(X, y, n_splits=5, max_depth=None, min_size=1, min_impurity_decrease=0.0, random_state=0): 
    kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    scores = []
    for train_index, test_index in kf.split(X):
        train_X, test_X = X.iloc[train_index], X.iloc[test_index]
        train_y, test_y = y.iloc[train_index], y.iloc[test_index]
        
        model = HandmadeDecisionTree(max_depth=max_depth, min_size=min_size, min_impurity_decrease=min_impurity_decrease)
        model.fit(train_X, train_y)
        scores.append(model.score(test_X, test_y))
    return np.mean(scores) 

max_depths = [1, 2, 3, 4, 5]
min_sizes = [1, 2, 5, 10]
min_impurity_decreases = [0.0, 0.001, 0.002, 0.005]


best_score = 0
best_params = {'max_depth': None, 'min_size': 1, 'min_impurity_decrease': 0.0}

for max_depth in max_depths:
    for min_size in min_sizes:
        for min_impurity_decrease in min_impurity_decreases:
            avg_score = cross_validate(X_train, y_train, n_splits=5, max_depth=max_depth, min_size=min_size, min_impurity_decrease=min_impurity_decrease)
            print(f"Depth: {max_depth}, Size: {min_size}, Impurity Decrease: {min_impurity_decrease}, Score: {avg_score}")
            if avg_score > best_score:
                best_score = avg_score
                best_params['max_depth'] = max_depth
                best_params['min_size'] = min_size
                best_params['min_impurity_decrease'] = min_impurity_decrease

print(f"Best Parameters: {best_params}, Best Score: {best_score}")



Depth: 1, Size: 1, Impurity Decrease: 0.0, Score: 0.625
Depth: 1, Size: 1, Impurity Decrease: 0.001, Score: 0.625
Depth: 1, Size: 1, Impurity Decrease: 0.002, Score: 0.625
Depth: 1, Size: 1, Impurity Decrease: 0.005, Score: 0.625
Depth: 1, Size: 2, Impurity Decrease: 0.0, Score: 0.625
Depth: 1, Size: 2, Impurity Decrease: 0.001, Score: 0.625
Depth: 1, Size: 2, Impurity Decrease: 0.002, Score: 0.625
Depth: 1, Size: 2, Impurity Decrease: 0.005, Score: 0.625
Depth: 1, Size: 5, Impurity Decrease: 0.0, Score: 0.625
Depth: 1, Size: 5, Impurity Decrease: 0.001, Score: 0.625
Depth: 1, Size: 5, Impurity Decrease: 0.002, Score: 0.625
Depth: 1, Size: 5, Impurity Decrease: 0.005, Score: 0.625
Depth: 1, Size: 10, Impurity Decrease: 0.0, Score: 0.625
Depth: 1, Size: 10, Impurity Decrease: 0.001, Score: 0.625
Depth: 1, Size: 10, Impurity Decrease: 0.002, Score: 0.625
Depth: 1, Size: 10, Impurity Decrease: 0.005, Score: 0.625
Depth: 2, Size: 1, Impurity Decrease: 0.0, Score: 0.5666666666666667
Depth: 

请注意，上面提到的参数搜索和模型评估方法是相对基础的。在实际应用中，可能会使用更复杂的方法（如网格搜索GridSearchCV或随机搜索RandomizedSearchCV），这些方法在机器学习库（如scikit-learn）中已有实现。不过，由于你提到想要从头开始实现决策树，上述方法是一个很好的起点。

In [42]:
model = HandmadeDecisionTree(max_depth=4, min_size=1)
model.fit(X_train, y_train)

# 对测试集进行预测
predictions = test_df.apply(model.predict_row, axis=1)

predictions = predictions.to_numpy()

# 输出预测结果
print(predictions)

[nan nan nan nan nan nan nan nan nan nan  3.  2.  2.  2.  2.  2.  2.  2.
  2.  2.  2. nan  3. nan nan nan nan nan nan nan]


In [43]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=4, min_samples_split=5)

# 训练模型
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


In [44]:
from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:")
print(classification_report(predictions, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(predictions, y_pred))

Classification Report:


/Users/kexueyi/miniforge3/envs/gaml/lib/python3.8/site-packages/sklearn/utils/_array_api.py:245: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y_true contains NaN.